In [33]:
from share import *
import config

import cv2
import einops
import gradio as gr
import numpy as np
import torch
import random

from pytorch_lightning import seed_everything
from annotator.util import resize_image, HWC3
from annotator.uniformer import UniformerDetector
from cldm.model import create_model, load_state_dict
from cldm.ddim_hacked import DDIMSampler

from pyqart import QrPainter, QrImagePrinter

apply_uniformer = UniformerDetector()

model = create_model('./models/cldm_v15.yaml').cpu()
model.load_state_dict(load_state_dict('./models/control_sd15_seg.pth', location='cuda'))
model = model.cuda()
ddim_sampler = DDIMSampler(model)


def process(prompt, a_prompt, n_prompt, num_samples, ddim_steps, guess_mode, strength, scale, seed, eta):
    with torch.no_grad():
        QR_VERSION = 5
        module_size = 20
        W = H = 512
        painter = QrPainter('http://www.nankai.edu.cn/', QR_VERSION)
        qrbin = QrImagePrinter.print(painter, point_width=1)
        qrbin = np.array(qrbin.convert('L'))
        qrbin = qrbin.repeat(module_size,axis=0).repeat(module_size, axis=1)
        qrbin = cv2.resize(qrbin, (W, H), interpolation=cv2.INTER_NEAREST)

        qr_anno = np.zeros((qrbin.shape[0], qrbin.shape[1], 3) )
        qr_anno[qrbin == 255] = np.array([180, 120, 120])
        qr_anno[qrbin == 0] = np.array([2, 226, 231])

        control = torch.from_numpy(qr_anno.copy()).float().cuda() / 255.0
        control = torch.stack([control for _ in range(num_samples)], dim=0)
        control = einops.rearrange(control, 'b h w c -> b c h w').clone()

        if seed == -1:
            seed = random.randint(0, 65535)
        seed_everything(seed)

        if config.save_memory:
            model.low_vram_shift(is_diffusing=False)

        cond = {"c_concat": [control], "c_crossattn": [model.get_learned_conditioning([prompt + ', ' + a_prompt] * num_samples)]}
        un_cond = {"c_concat": None if guess_mode else [control], "c_crossattn": [model.get_learned_conditioning([n_prompt] * num_samples)]}
        shape = (4, H // 8, W // 8)

        if config.save_memory:
            model.low_vram_shift(is_diffusing=True)

        model.control_scales = [strength * (0.825 ** float(12 - i)) for i in range(13)] if guess_mode else ([strength] * 13)  # Magic number. IDK why. Perhaps because 0.825**12<0.01 but 0.826**12>0.01
        samples, intermediates = ddim_sampler.sample(ddim_steps, num_samples,
                                                     shape, cond, verbose=False, eta=eta,
                                                     unconditional_guidance_scale=scale,
                                                     unconditional_conditioning=un_cond)

        if config.save_memory:
            model.low_vram_shift(is_diffusing=False)

        x_samples = model.decode_first_stage(samples)
        x_samples = (einops.rearrange(x_samples, 'b c h w -> b h w c') * 127.5 + 127.5).cpu().numpy().clip(0, 255).astype(np.uint8)

        results = [x_samples[i] for i in range(num_samples)]
    return [qr_anno] + results

True

In [35]:
qrbin

array([[255, 255, 255, ..., 255, 255, 255],
       [255, 255, 255, ..., 255, 255, 255],
       [255, 255, 255, ..., 255, 255, 255],
       ...,
       [255, 255, 255, ..., 255, 255, 255],
       [255, 255, 255, ..., 255, 255, 255],
       [255, 255, 255, ..., 255, 255, 255]], dtype=uint8)

In [38]:
qr_anno = np.zeros( (qrbin.shape[0], qrbin.shape[1], 3) )
qr_anno[qrbin == 255] = np.array([230, 230, 6])
qr_anno[qrbin == 0] = np.array([3, 200, 4])

In [40]:
qr_anno

array([[[230., 230.,   6.],
        [230., 230.,   6.],
        [230., 230.,   6.],
        ...,
        [230., 230.,   6.],
        [230., 230.,   6.],
        [230., 230.,   6.]],

       [[230., 230.,   6.],
        [230., 230.,   6.],
        [230., 230.,   6.],
        ...,
        [230., 230.,   6.],
        [230., 230.,   6.],
        [230., 230.,   6.]],

       [[230., 230.,   6.],
        [230., 230.,   6.],
        [230., 230.,   6.],
        ...,
        [230., 230.,   6.],
        [230., 230.,   6.],
        [230., 230.,   6.]],

       ...,

       [[230., 230.,   6.],
        [230., 230.,   6.],
        [230., 230.,   6.],
        ...,
        [230., 230.,   6.],
        [230., 230.,   6.],
        [230., 230.,   6.]],

       [[230., 230.,   6.],
        [230., 230.,   6.],
        [230., 230.,   6.],
        ...,
        [230., 230.,   6.],
        [230., 230.,   6.],
        [230., 230.,   6.]],

       [[230., 230.,   6.],
        [230., 230.,   6.],
        [230., 2

In [29]:
np.sum(black_mask)

40621500

In [18]:
qrbin[qrbin == [255, 255, 255]] = [230, 230, 6]

ValueError: NumPy boolean array indexing assignment cannot assign 3 input values to the 548775 output values where the mask is true

In [41]:
import cv2 
seg = cv2.imread('seg.jpg')

In [43]:
seg.shape

(512, 512, 3)